In [1]:
# Standard modules
import pdb
import os
import sys
import pandas as pd
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import gridspec

from lmfit import Parameters, minimize, fit_report

import warnings
warnings.filterwarnings("ignore")

sys.path.append("../../../..")

from simstackwrapper import SimstackWrapper
from simstackresults import SimstackResults
from simstackplots import SimstackPlots
from simstacktoolbox import SimstackToolbox

#from scipy.signal import find_peaks
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [19]:
# Import saved stacking results
combined_pickle_filename = "cosmos2020_farmer_nuvrj_0p01_0p5_1_1p5_2_2p5_3_3p5_4_5_6_8_10p0_X_4_foregnd_atonce_bootstrap_1-150"
combined_pickle_path = os.environ['PICKLESPATH']+'/'+os.path.join("simstack","stacked_flux_densities","spire", combined_pickle_filename)
underscore_suffix = ""
path_pickle_file = os.path.join(combined_pickle_path, combined_pickle_filename+".pkl")
simstack_object = SimstackToolbox.import_saved_pickles(path_pickle_file)

In [2]:
# Import catalog with SimstackWrapper
combined_pickle_filename = "cosmos2020_farmer_nuvrj_0p01_0p5_1_1p5_2_2p5_3_3p5_4_5_6_8_10p0_X_4_foregnd_atonce_bootstrap_1-150"
combined_pickle_path = os.environ['PICKLESPATH']+'/'+os.path.join("simstack","stacked_flux_densities", "spire" , combined_pickle_filename)
underscore_suffix = ""
path_ini_file = os.path.join(combined_pickle_path, "cosmos2020_farmer.ini")
path_pickle_file = os.path.join(combined_pickle_path, combined_pickle_filename+".pkl")
if os.path.isfile(path_ini_file):
    simstack_catalog_object = SimstackWrapper(path_ini_file, read_maps=False, read_catalog=True, keep_catalog=True, stack_automatically=False, save_automatically=False)
else:
    print(path_ini_file+' not found')

# Import Catalog and PZ to link objects and their photometric redshift PDFs

### SIDES Catalog

In [3]:
# Import catalog with SimstackWrapper
combined_pickle_filename = "sides_allz_nuvrj_0p01_0p5_1_1p5_2_2p5_3_4_5_6p0_X_4_foregnd_atonce_bootstrap_1-70"
combined_pickle_path = os.environ['PICKLESPATH']+'/'+os.path.join("simstack","stacked_flux_densities", "spire" , combined_pickle_filename)
underscore_suffix = ""
path_ini_file = os.path.join(combined_pickle_path, "sides_simulation.ini")
path_pickle_file = os.path.join(combined_pickle_path, combined_pickle_filename+".pkl")
if os.path.isfile(path_ini_file):
    sides_catalog_object = SimstackWrapper(path_ini_file, read_maps=False, read_catalog=True, keep_catalog=True, stack_automatically=False, save_automatically=False)
else:
    print(path_ini_file+' not found')

In [4]:
sides_catalog_object.catalog_dict['tables']['full_table'].tail()

,Unnamed: 0,redshift,ra,dec,Mstar,qflag,SFR,LIR,SMIPS24,S100,S160,SSPIRE250,SSPIRE350,SSPIRE500,S850,Trf,Ain
361480,361480,8.601702,0.416990,0.015490,1.572414e+09,False,13.236866,1.323687e+11,1.203157e-08,7.719299e-07,0.000003,0.000006,0.000027,0.000054,0.000064,39.307918,-33.932404
361481,361481,8.566001,0.239313,0.308510,2.291832e+09,False,19.834282,1.983428e+11,2.547088e-08,1.650851e-06,0.000006,0.000013,0.000053,0.000108,0.000140,39.689883,-33.670603
361482,361482,8.648121,0.974505,1.053851,4.165142e+09,False,14.366696,1.436670e+11,1.746845e-08,1.093338e-06,0.000004,0.000009,0.000035,0.000073,0.000098,40.874420,-33.904004
361483,361483,8.732985,0.035478,1.327398,3.519983e+09,False,15.789969,1.578997e+11,1.082463e-08,6.367038e-07,0.000003,0.000005,0.000021,0.000045,0.000058,29.076731,-33.086036
361484,361484,8.719206,0.866146,0.510229,1.293671e+09,False,38.969204,3.896920e+11,4.499536e-08,2.662770e-06,0.000011,0.000022,0.000091,0.000188,0.000239,41.175243,-33.491933


### COSMOS2020 Catalog

In [5]:
# Load Table into a pandas DataFrame
cosmos2020 = simstack_catalog_object.catalog_dict['tables']['full_table']

In [6]:
cosmos2020.tail()

,ID,ALPHA_J2000,DELTA_J2000,lp_zBEST,lp_mass_med,ez_restU-V,ez_restV-J,restNUV-R,restR-J,lp_SFR_best,lp_sSFR_med,lp_type,sfg
711880,964439,150.212106,1.787928,0.3207,7.49669,NaN,NaN,1.24270,-0.08177,-1.7330,-8.97360,0,1.0
711881,964440,150.211415,1.788014,0.0490,6.65096,NaN,NaN,6.23957,1.09682,-5.9900,-12.76579,0,0.0
711882,964441,150.212666,1.788418,0.8165,7.61975,NaN,NaN,0.84868,-0.24417,-0.7851,-8.43227,0,1.0
711883,964444,150.212557,1.789332,0.0812,5.16792,NaN,NaN,0.90254,-1.10197,-2.1850,-7.30000,0,1.0
711884,964449,150.201013,1.790721,0.3073,7.28359,NaN,NaN,0.66499,-0.43547,-1.2590,-8.51345,0,1.0


### P(z) 

In [7]:
# Import table with astropy
path_cosmos2020_pz = os.path.join(os.environ['CATSPATH'], "cosmos","PZ","COSMOS2020_FARMER_R1_v2.0_LEPHARE_PZ.fits")
if os.path.isfile(path_cosmos2020_pz):
    print('Open ',path_cosmos2020_pz)
    with fits.open(path_cosmos2020_pz) as hdul_pz:
        hdul_pz.verify('fix')
        cosmos_catalog_pz = hdul_pz[0].data
else:
    print(path_cosmos2020_pz, ' not found')

Open  D:\catalogs\cosmos\PZ\COSMOS2020_FARMER_R1_v2.0_LEPHARE_PZ.fits


In [8]:
# Load Table into a pandas DataFrame
cosmos2020_pz = pd.DataFrame(cosmos_catalog_pz)

In [9]:
cosmos2020_pz.tail()

,0,1,2,3,4,5,6,7,8,9,...,992,993,994,995,996,997,998,999,1000,1001
964502,964502.0,0.0,0.000000,0.010244,0.011078,0.011789,0.012833,0.014263,0.018287,0.044225,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000
964503,964503.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000
964504,964504.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000
964505,964505.0,0.0,0.007243,0.008699,0.010084,0.011576,0.012597,0.009745,0.008120,0.014568,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000
964506,964506.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001795,0.001811,0.001828,0.001844,0.00186,0.001876,0.001891,0.001906,0.00192,0.001933


In [10]:
def black( nu_in, T):
    # h = 6.623e-34     ; Joule*s
    # k = 1.38e-23      ; Joule/K
    # c = 3e8           ; m/s
    # (2*h*nu_in^3/c^2)*(1/( exp(h*nu_in/k*T) - 1 )) * 10^29

    a0 = 1.4718e-21  # 2*h*10^29/c^2
    a1 = 4.7993e-11  # h/k

    num = a0 * nu_in ** 3.0
    den = np.exp(a1 * np.outer(1.0 / T, nu_in)) - 1.0
    ret = num / den

    return ret

In [11]:
def graybody_fn(theta, x, alphain=2.0, betain=1.8):
    A, T = theta

    c_light = 299792458.0  # m/s

    nu_in = np.array([c_light * 1.e6 / wv for wv in x])
    ng = np.size(A)

    base = 2.0 * (6.626) ** (-2.0 - betain - alphain) * (1.38) ** (3. + betain + alphain) / (2.99792458) ** 2.0
    expo = 34.0 * (2.0 + betain + alphain) - 23.0 * (3.0 + betain + alphain) - 16.0 + 26.0
    K = base * 10.0 ** expo
    w_num = 10 ** A * K * (T * (3.0 + betain + alphain)) ** (3.0 + betain + alphain)
    w_den = (np.exp(3.0 + betain + alphain) - 1.0)
    w_div = w_num / w_den
    nu_cut = (3.0 + betain + alphain) * 0.208367e11 * T
    graybody = np.reshape(10 ** A, (ng, 1)) * nu_in ** np.reshape(betain, (ng, 1)) * black(nu_in, T) / 1000.0
    powerlaw = np.reshape(w_div, (ng, 1)) * nu_in ** np.reshape(-1.0 * alphain, (ng, 1))
    graybody[np.where(nu_in >= np.reshape(nu_cut, (ng, 1)))] = \
        powerlaw[np.where(nu_in >= np.reshape(nu_cut, (ng, 1)))]

    return graybody

In [12]:
def fast_sed(m, wavelengths):

    v = m.valuesdict()
    A = np.asarray(v['A'])
    T = np.asarray(v['T_observed'])
    betain = np.asarray(v['beta'])
    alphain = np.asarray(v['alpha'])
    theta_in = A, T

    return graybody_fn(theta_in, wavelengths, alphain=alphain, betain=betain)

In [13]:
def find_sed_min(params, wavelengths, fluxes, covar=None):

    graybody = fast_sed(params, wavelengths)[0]
    delta_y = (fluxes - graybody)

    if (covar is None) or (np.sum(covar) == 0):
        return delta_y
    else:
        if np.shape(covar) == np.shape(fluxes):
            return delta_y ** 2 / covar
        else:
            return np.matmul(delta_y**2, np.linalg.inv(covar))

In [14]:
def fast_sed_fitter(wavelengths, fluxes, covar=None, betain=1.8, alphain=2.0, redshiftin=0, stellarmassin=None):

    t_in = (23.8 + 2.7 * redshiftin + 0.9 * redshiftin ** 2) / (1 + redshiftin)
    if stellarmassin is not None:
        a_in = -47 - redshiftin*0.05 + 11 * (stellarmassin / 10)
    else:
        a_in = -35.0
    #print(t_in, a_in)
    fit_params = Parameters()
    fit_params.add('A', value=a_in, vary=True)
    fit_params.add('T_observed', value=t_in, vary=True)
    fit_params.add('beta', value=betain, vary=False)
    fit_params.add('alpha', value=alphain, vary=False)

    fluxin = fluxes #[np.max([i, 1e-7]) for i in fluxes]
    try:
        sed_params = minimize(find_sed_min, fit_params,
                              args=(wavelengths,),
                              kws={'fluxes': fluxin, 'covar': covar})
        m = sed_params.params
    except:
        m = fit_params

    return m

In [16]:
def estimate_single_sed(wavelengths, fluxes, z=0, m=None):

    sed_params = fast_sed_fitter(wavelengths, fluxes, covar=None, redshiftin=z, stellarmassin=m)
    
    return sed_params

In [17]:
def mean_sed_from_pz_full_simulation(self, catalog, sides, nsims=2, wv_cat = ["SMIPS24","S100","S160","SSPIRE250","SSPIRE350","SSPIRE500"], wavelengths = [24, 100, 160, 250, 350, 500]):

    bin_keys = list(self.config_dict['parameter_names'].keys())    
    zlen = len(self.config_dict['parameter_names'][bin_keys[0]])
    mlen = len(self.config_dict['parameter_names'][bin_keys[1]])
    plen = len(self.config_dict['parameter_names'][bin_keys[2]])

    gal_ids = {}
    fluxes_best = {}
    fluxes_drawn = {}
    tdust_best = {}
    tdust_drawn = {}
    redshifts_best = {}
    redshifts_drawn = {}
    ave_theta_best = {}
    ave_theta_drawn = {}
    ngals_bin = {}
    full_sim_dict = {'wavelengths': wavelengths, 'bin_dict': self.config_dict['parameter_names'],
                     'sbest': fluxes_best, 'sdrawn': fluxes_drawn, 'tbest': tdust_best, 'tdrawn':tdust_drawn, 
                     'theta_best':ave_theta_best, 'theta_drawn':ave_theta_drawn, 'ids': gal_ids,
                     'zbest': redshifts_best, 'zdrawn': redshifts_drawn, 'ngals': ngals_bin}
    
    for iz, zlab in enumerate(self.config_dict['parameter_names'][bin_keys[0]]):
        for ip, plab in enumerate(self.config_dict['parameter_names'][bin_keys[2]]):
            if ip:
                for im, mlab in enumerate(self.config_dict['parameter_names'][bin_keys[1]]):
                    id_label = "__".join([zlab, mlab, plab])
                    #print(id_label)

                    m = [float(i) for i in mlab.split('_')[2:]]
                    z = [float(i) for i in zlab.split('_')[1:]]

                    #pdb.set_trace()
                    ind_bin = (catalog['lp_zBEST'] > z[0]) & \
                    (catalog['lp_zBEST'] < z[1]) & \
                    (catalog['lp_mass_med'] >= m[0]) & \
                    (catalog['lp_mass_med'] < m[1]) & \
                    (catalog['sfg'] == ip)
                    idz=catalog.ID[ind_bin].values

                    # Find all sfg in a mass range
                    subsides = sides[(sides['Mstar']>=10**m[0]) & (sides['Mstar']<=10**m[1]) & (sides['qflag']==0)]
                    # Find all sfg in a mass and redshift range
                    subsides_bin = sides[(sides['redshift']>=z[0]) & (sides['redshift']<z[1]) & (sides['Mstar']>=10**m[0]) & (sides['Mstar']<10**m[1]) & (sides['qflag']==0)]
                    
                    flux_bin = np.zeros([nsims, len(idz),len(wv_cat)])
                    flux_bin0= np.zeros([len(idz),len(wv_cat)])
                    a_cat0 = np.zeros(len(idz))
                    a_cat = np.zeros([nsims, len(idz)])
                    trf_cat0 = np.zeros(len(idz))
                    trf_cat = np.zeros([nsims, len(idz)])
                    z_drawn = np.zeros([nsims, len(idz)])
                    z_best = catalog['lp_zBEST'].loc[ind_bin]
                    gal_id_bin = np.zeros([nsims, len(idz)])
                    
                    for i in range(len(idz)):
                        cin = cosmos2020_pz[0] == idz[i]
                        indz = cosmos2020_pz[0][cin]
                        
                        # BEST-z
                        try:
                            dz = abs(subsides['redshift'] - z_best.values[i])
                            if np.min(dz) < 0.1:
                                idx_sides_0 = np.argmin(dz)
                                flux_bin0[i] = subsides.iloc[idx_sides_0][wv_cat].values
                                a_cat0[i] = subsides.iloc[idx_sides_0]['Ain'] 
                                trf_cat0[i] = subsides.iloc[idx_sides_0]['Trf'] 
                            else:
                                print('No galaxies at {0:0.1f}'.format(draw_redshift))
                        except:
                            #print('Fcuk')
                            pass
                        
                        # DRAWN-z
                        pz = cosmos2020_pz.loc[int(indz.values)][1:].astype('float64')/np.sum(cosmos2020_pz.loc[int(indz.values)][1:].astype('float64'))
                        for isim in range(nsims):
                            np.random.seed(isim)
                            try:
                                draw_redshift = np.random.choice(cosmos2020_pz.iloc[0][1:],p=pz)
                                dz = abs(subsides['redshift'] - draw_redshift)
                                if np.min(dz) < 0.1:
                                    idx_sides_drawn = np.argmin(dz)
                                    flux_bin[isim, i] = subsides.iloc[idx_sides_drawn][wv_cat].values
                                    a_cat[isim, i] = subsides.iloc[idx_sides_drawn]['Ain'] 
                                    trf_cat[isim, i] = subsides.iloc[idx_sides_drawn]['Trf'] 
                                    z_drawn[isim, i] = draw_redshift
                                    gal_id_bin[isim, i] = idx_sides_drawn
                                else:
                                    print('No galaxies at {0:0.1f}'.format(draw_redshift))
                            except:
                                #pdb.set_trace()
                                #print('whoopsie')
                                pass

                    ave_flux0 = np.mean(flux_bin0, axis=0)
                    ave_flux = np.mean(flux_bin,axis=1)
                    fluxes_best[id_label] = ave_flux0
                    fluxes_drawn[id_label] = ave_flux
                    tdust_best[id_label] = trf_cat0
                    tdust_drawn[id_label] = trf_cat
                    redshifts_best[id_label] = z_best
                    redshifts_drawn[id_label] = z_drawn
                    ngals_bin[id_label] = len(idz)
                    
                    # Estimate SED Tdust
                    single_sed_params0 = estimate_single_sed(wavelengths, ave_flux0)
                    ave_theta_best[id_label] = [single_sed_params0['A'].value, single_sed_params0['T_observed'].value]
                    
                    ave_theta_drawn[id_label] = np.zeros([nsims, 2])
                    for isim2 in range(nsims):
                        single_sed_params = estimate_single_sed(wavelengths, ave_flux[isim2])
                        ave_theta_drawn[id_label][isim2] = single_sed_params['A'].value, single_sed_params['T_observed'].value

                    #pdb.set_trace()

    return full_sim_dict

In [ ]:
nsims = 10
sed_sim_100 = mean_sed_from_pz_full_simulation(simstack_object, cosmos2020, sides_catalog_object.catalog_dict['tables']['full_table'], nsims=nsims)

In [ ]:
# Save Total LIR Density results
sed_sim_filename =  '_'.join(['sed_simulation',str(nsims)])+'.pkl'
SimstackToolbox.save_to_pickles(os.path.join(combined_pickle_path,sed_sim_filename), sed_sim_100)
print('Saving', os.path.join(combined_pickle_path,sed_sim_filename))

In [ ]:
def plot_mean_sed_from_pz_full_simulation(self, sim_dict):
    
    bin_keys = list(sim_dict['bin_dict'].keys())
    wavelengths = sim_dict['wavelengths']
    #bin_keys = list(self.config_dict['parameter_names'].keys())    
    zlen = len(self.config_dict['parameter_names'][bin_keys[0]])
    mlen = len(self.config_dict['parameter_names'][bin_keys[1]])
    plen = len(self.config_dict['parameter_names'][bin_keys[2]])
    
    width_ratios = [i for i in np.ones(zlen)]
    gs = gridspec.GridSpec(mlen, zlen, width_ratios=width_ratios,
                           wspace=0.0, hspace=0.0, top=0.95, bottom=0.05, left=0.17, right=0.845)
    fig = plt.figure(figsize=(41, 11))
    
    fluxes_best = sim_dict['sbest']
    fluxes_drawn = sim_dict['sdrawn']
    tdust_best = sim_dict['tbest']
    tdust_drawn = sim_dict['tdrawn']
    redshifts_best = sim_dict['zbest']
    redshifts_drawn = sim_dict['zdrawn']
    ave_theta_best = sim_dict['theta_best']
    ave_theta_drawn = sim_dict['theta_drawn']
    ngals_bin = sim_dict['ngals']
        
    wv_mod = np.linspace(8,1000,1000)
    for iz, zlab in enumerate(self.config_dict['parameter_names'][bin_keys[0]]):
        for ip, plab in enumerate(self.config_dict['parameter_names'][bin_keys[2]]):
            if ip:
                for im, mlab in enumerate(self.config_dict['parameter_names'][bin_keys[1]]):
                    id_label = "__".join([zlab, mlab, plab])
                    ngals = ngals_bin[id_label]

                    m = [float(i) for i in mlab.split('_')[2:]]
                    z = [float(i) for i in zlab.split('_')[1:]]
                    
                    ax = plt.subplot(gs[3-im, iz])
                    ax.set_xscale('log')
                    ax.set_yscale('log')
                    ax.set_ylim([1e-5,1e-1])
                    ax.set_title(zlab.replace('redshift_','z=').replace('_','-'))

                    nsims = len(fluxes_drawn[id_label])
                    tdust_drawn_sim = np.zeros(nsims)
                    for isim in range(nsims)[::-1]:
                        ave_flux = fluxes_drawn[id_label][isim]                        
                        ax.scatter(wavelengths, ave_flux, c='r')
                        
                        theta_in = ave_theta_drawn[id_label][isim]
                        tdust_drawn_sim[isim] = theta_in[1] * (1+np.mean(z))
                        
                        graybody = graybody_fn(theta_in, wv_mod)[0]
                        
                        if not isim:
                            label='drawn:{0:0.1f}+-{1:0.1f}K'.format(np.mean(tdust_drawn_sim),np.std(tdust_drawn_sim))
                            ax.plot(wv_mod, graybody, label=label, c='r', lw=0.5)
                        else:
                            ax.plot(wv_mod, graybody, c='r', lw=0.5)
                    
                    ave_flux0 = fluxes_best[id_label]
                    ax.scatter(wavelengths, ave_flux0, c='b')

                    theta_in = ave_theta_best[id_label]
                    graybody = graybody_fn(theta_in, wv_mod)[0]
                    label='best={0:0.1f}K'.format(theta_in[1] * (1+np.mean(z)))
                    ax.plot(wv_mod, graybody, label=label, c='b')
                    ax.legend()
                    #pdb.set_trace()

                    if im:
                        ax.set_xticklabels([])
                    else:
                        ax.set_xlabel('wavelength')

                    if iz:
                        ax.set_yticklabels([])
                    else:
                        ax.set_ylabel('Flux')

                    if iz == zlen - 1:
                        ax.yaxis.set_label_position("right")
                        ax.set_ylabel(mlab.replace('stellar_mass_', 'log(M)=').replace('_', '-'))

In [ ]:
def plot_redshift_hist_from_pz_full_simulation(self, sim_dict): 
    
    color = ['b','y','g','r','m']
    bin_keys = list(sim_dict['bin_dict'].keys())
    wavelengths = sim_dict['wavelengths']
    #bin_keys = list(self.config_dict['parameter_names'].keys())    
    zlen = len(self.config_dict['parameter_names'][bin_keys[0]])
    mlen = len(self.config_dict['parameter_names'][bin_keys[1]])
    plen = len(self.config_dict['parameter_names'][bin_keys[2]])
    
    width_ratios = [i for i in np.ones(zlen)]
    gs = gridspec.GridSpec(mlen, zlen, width_ratios=width_ratios,
                           wspace=0.0, hspace=0.0, top=0.95, bottom=0.05, left=0.17, right=0.845)
    fig = plt.figure(figsize=(40, 12))
    
    fluxes_best = sim_dict['sbest']
    fluxes_drawn = sim_dict['sdrawn']
    tdust_best = sim_dict['tbest']
    tdust_drawn = sim_dict['tdrawn']
    redshifts_best = sim_dict['zbest']
    redshifts_drawn = sim_dict['zdrawn']
    ave_theta_best = sim_dict['theta_best']
    ave_theta_drawn = sim_dict['theta_drawn']
    ngals_bin = sim_dict['ngals']
        
    wv_mod = np.linspace(8,1000,1000)
    for iz, zlab in enumerate(self.config_dict['parameter_names'][bin_keys[0]]):
        for ip, plab in enumerate(self.config_dict['parameter_names'][bin_keys[2]]):
            if ip:
                for im, mlab in enumerate(self.config_dict['parameter_names'][bin_keys[1]]):
                    id_label = "__".join([zlab, mlab, plab])
                    ngals = ngals_bin[id_label]

                    m = [float(i) for i in mlab.split('_')[2:]]
                    z = [float(i) for i in zlab.split('_')[1:]]
                    
                    ax = plt.subplot(gs[3-im, iz])
                    ax.set_title(zlab.replace('redshift_','z=').replace('_','-'))

                    nsims = len(fluxes_drawn[id_label])

                    hb = np.linspace(0,10,51)
                        
                    for isim in range(nsims)[::-1]:
                        #pdb.set_trace()
                        z_drawn = redshifts_drawn[id_label][isim]     
                        hy, hx = np.histogram(z_drawn, bins=hb)
                        
                        if not isim and not iz:
                            label='drawn'
                            ax.step((hx[:-1]+hx[1:])/2, hy/1, c='r',lw=0.5, label=label)
                        else:
                            ax.step((hx[:-1]+hx[1:])/2, hy/1, c='r',lw=0.5)
                    
                    z_best = redshifts_best[id_label]
                    hy, hx = np.histogram(z_best, bins=hb)
                    label='best'
                    if not iz: 
                        ax.step((hx[:-1]+hx[1:])/2, hy, c='b',lw=1.5, label=label)
                    else:
                        ax.step((hx[:-1]+hx[1:])/2, hy, c='b',lw=1.5)

                    if not iz: 
                        ax.legend()

                    if im:
                        ax.set_xticklabels([])
                    else:
                        ax.set_xlabel('redshift')

                    if iz:
                        ax.set_yticklabels([])

                    if iz == zlen - 1:
                        ax.yaxis.set_label_position("right")
                        ax.set_ylabel(mlab.replace('stellar_mass_', 'log(M)=').replace('_', '-'))

In [49]:
def plot_tdust_hist_from_pz_full_simulation(self, sim_dict): 
    
    color = ['b','y','g','r','m']
    bin_keys = list(sim_dict['bin_dict'].keys())
    wavelengths = sim_dict['wavelengths']
    zlen = len(self.config_dict['parameter_names'][bin_keys[0]])
    mlen = len(self.config_dict['parameter_names'][bin_keys[1]])
    plen = len(self.config_dict['parameter_names'][bin_keys[2]])
    
    width_ratios = [i for i in np.ones(zlen)]
    gs = gridspec.GridSpec(mlen, zlen, width_ratios=width_ratios,
                           wspace=0.0, hspace=0.0, top=0.95, bottom=0.05, left=0.17, right=0.845)
    fig = plt.figure(figsize=(40, 12))
    
    fluxes_best = sim_dict['sbest']
    fluxes_drawn = sim_dict['sdrawn']
    tdust_best = sim_dict['tbest']
    tdust_drawn = sim_dict['tdrawn']
    redshifts_best = sim_dict['zbest']
    redshifts_drawn = sim_dict['zdrawn']
    ave_theta_best = sim_dict['theta_best']
    ave_theta_drawn = sim_dict['theta_drawn']
    ngals_bin = sim_dict['ngals']
        
    wv_mod = np.linspace(8,1000,1000)
    for iz, zlab in enumerate(self.config_dict['parameter_names'][bin_keys[0]]):
        for ip, plab in enumerate(self.config_dict['parameter_names'][bin_keys[2]]):
            if ip:
                for im, mlab in enumerate(self.config_dict['parameter_names'][bin_keys[1]]):
                    id_label = "__".join([zlab, mlab, plab])
                    ngals = ngals_bin[id_label]

                    m = [float(i) for i in mlab.split('_')[2:]]
                    z = [float(i) for i in zlab.split('_')[1:]]
                    
                    ax = plt.subplot(gs[3-im, iz])
                    ax.set_title(zlab.replace('redshift_','z=').replace('_','-'))

                    nsims = len(fluxes_drawn[id_label])

                    hb = np.linspace(20,70,26)
                        
                    for isim in range(nsims)[::-1]:
                        #pdb.set_trace()
                        t_drawn = tdust_drawn[id_label][isim]     
                        hy, hx = np.histogram(t_drawn, bins=hb)
                        
                        if not isim and not iz:
                            label='drawn'
                            ax.step((hx[:-1]+hx[1:])/2, hy/1, c='r',lw=0.5, label=label)
                        else:
                            ax.step((hx[:-1]+hx[1:])/2, hy/1, c='r',lw=0.5)
                    
                    t_best = tdust_best[id_label]
                    hy, hx = np.histogram(t_best, bins=hb)
                    label='best'
                    if not iz: 
                        ax.step((hx[:-1]+hx[1:])/2, hy, c='b',lw=1.5, label=label)
                    else:
                        ax.step((hx[:-1]+hx[1:])/2, hy, c='b',lw=1.5)

                    if not iz: 
                        ax.legend()

                    if im:
                        ax.set_xticklabels([])
                    else:
                        ax.set_xlabel('Tdust')

                    if iz:
                        ax.set_yticklabels([])

                    if iz == zlen - 1:
                        ax.yaxis.set_label_position("right")
                        ax.set_ylabel(mlab.replace('stellar_mass_', 'log(M)=').replace('_', '-'))

In [ ]:
plot_mean_sed_from_pz_full_simulation(simstack_object, sed_sim_100)

In [ ]:
plot_redshift_hist_from_pz_full_simulation(simstack_object, sed_sim_100)

In [ ]:
plot_tdust_hist_from_pz_full_simulation(simstack_object, sed_sim_100)